In [3]:
import gadgets
import datasets
import transformers
import pandas as pd
import os

In [4]:
ds = datasets.load_dataset("gsm8k", "main")

In [5]:
calc = gadgets.gadget.Calculator()

os.makedirs("../data/gsm8k", exist_ok=True)
df = {}
for split in ["train", "test"]:
    df[split] = ds[split].to_pandas()
    examples = df[split].apply(gadgets.gsm8k.parse, axis=1)
    df[split].drop(columns=["answer"], inplace=True)
    df[split]["chain"] = examples.apply(lambda x: gadgets.markup.to_model_markup(example=x)).apply(str)
    df[split]["result"] = examples.apply(lambda x: x.result)
    df[split]["result_float"] = df[split]["result"].apply(calc._float_eval)
    df[split].to_json(f"../data/gsm8k/{split}.jsonl", orient="records", lines=True, force_ascii=False)

/home/xkadlci2/gadgets/gadgets/markup.py:14: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return bs4.BeautifulSoup(step, features="html.parser")
/home/xkadlci2/gadgets/gadgets/markup.py:14: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return bs4.BeautifulSoup(step, features="html.parser")


In [6]:
with open("../data/gsm8k/train.jsonl") as f:
    content = f.read()
with open("../data/gsm8k/test.jsonl") as f:
    content += f.read()
content.count('\u2028')

0

In [7]:
foo = pd.Series([x for x in content if not x.isascii()]).value_counts().reset_index()

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained("t5-base")
# TODO many random unicode characters that might not be in the tokenizer's vocab

In [ ]:
foo["tokenized"] = foo["index"].apply(tokenizer.encode).apply(tokenizer.decode, skip_special_tokens=True)

In [ ]:
foo